In [1]:
!pip install --upgrade plotly

     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 56 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=10054bc6c12d3412768a732618fa4c5ea0d1ae3b85f021f38ce9d6992990fb41
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [4]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep=',', index=False)

In [6]:
excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

In [7]:
athletes = pd.read_csv("Athletes.csv")
coaches = pd.read_csv("Coaches.csv")
entriesGender = pd.read_csv("EntriesGender.csv")
medals = pd.read_csv("Medals.csv")
teams = pd.read_csv("Teams.csv")

In [8]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [9]:
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [15]:
import plotly.express as px
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence=['Gold', 'Silver', 'Brown'], title="Number of medals per country" )

In [16]:
fig.show()

In [19]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [22]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from coaches').show()

+--------------------+--------------------+-----------------+--------+
|                Name|                 NOC|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [27]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as count from Coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head()

,NOC,count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1


In [29]:
fig = px.bar(pd_coaches_query, x='NOC', y='count', color='count', title='Number of coaches')
fig.show()

In [32]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from Athletes order by NOC, Discipline').show()

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [64]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from Athletes group by NOC, Discipline
             order by NOC, Discipline''').toPandas()

In [65]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of athletes from each country per discipline')
fig.show()

In [66]:
athletes['Discipline'].value_counts().head(10)

Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: Discipline, dtype: int64

In [67]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming','Football'])]
pd_athletes_query.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [68]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of athletes from each country per discipline')
fig.show()

In [69]:
entriesGender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [70]:
len(entriesGender)

46